In [1]:
from numpy import *
from matplotlib.pyplot import *
import scipy.linalg
import matplotlib.lines as mlines
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize']= 16,5
from __future__ import division

In [2]:
import pandas as pd
import csv
import os
import time
import gc

In [267]:
def load_data():
    """
Function to load data
    :return: data vector, data target, data mean, data standar deviation 
    """
    data = np.genfromtxt('Resources/data.csv', delimiter=',')
    d_mean = np.mean(data)
    d_std = np.std(data)
    valtarget = data[-12:-6]
    data = (data - d_mean) / d_std    
    return data, valtarget, d_mean, d_std

In [268]:
def load_dataTest():
    """
Function to load data
    :return: data vector, data target, data mean, data standar deviation 
    """
    data = np.genfromtxt('Resources/data.csv', delimiter=',')
    d_mean = np.mean(data)
    d_std = np.std(data)
    target = data[-6:]
    data = (data - d_mean) / d_std    
    return data, target, d_mean, d_std

In [4]:
def get_TS():
    rawData = pd.read_csv('Resources/Consulta_Banxico.csv', sep=',', encoding='latin1', skiprows=15, skip_blank_lines=True)
    Data = rawData.drop(rawData.index[:2])
    Data = Data.reset_index(drop = True)
    Data.columns = ['Fecha' , 'Tasa de crecimiento Mensual']
    TS = pd.Series(Data['Tasa de crecimiento Mensual'].astype(float))
    TS.index= Data['Fecha']
    del Data
    return TS

In [299]:
def generate_win_matrix(inSize, resSize, scaleFactor=1):
    """
Function to generate W in matrix with a uniform distribution between [-1,1]
in other Function the uniform distribution can be moved to any range
    :param inSize: number of inputs without interceptor
    :param resSize: size of reservoir
    :param scaleFactor: Scale Factor for the matrix by default 1
    :return: W in Matrix
    """
    Win = (random.uniform(low=-0.5, high=0.5, size=(resSize, 1 + inSize))) * scaleFactor
    return Win

In [298]:
def generate_w_matrix(resSize):
    """
Function to generate W matrix with a uniform distribution between [-1,1]
 In other function the uniform distribution can be moved to any range
    :param resSize: size of reservoir
    :return: W matrix
    """
    W = random.uniform(low=-0.5, high= 0.5, size = (resSize,resSize))
    return  W

In [7]:
def get_spectral_radius(W):
    """
Function to get the spectral radius of W matrix
    :param W: W matrix
    :return: spectral radius
    """
    rhoW = max(abs(linalg.eig(W)[0]))
    return  rhoW

In [8]:
def scale_w(W,rhoW, SRS = 2):
    """
Function to apply spectral radius scale factor
    :param W: W matrix
    :param rhoW: spectral radius
    :param SRS: spectral radius scale factor by default 1
    :return: W
    """
    W*= (SRS/ rhoW)
    return W

In [9]:
def get_x_yt_matrix(data,inSize, resSize,trainLen,initLen):
    """
Function to get X and Yt Matrix to save activation states and prediction targets
    :type data: normalized data
    :param inSize: size of the input 
    :param resSize: size of reservoir
    :param trainLen: length of train data
    :param initLen:  length of initial data
    :return: X, Yt
    """
    X = zeros((1+inSize+resSize,trainLen-initLen))
    Yt = data[None,initLen+1:trainLen+1]
    return X,Yt

In [10]:
def get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X):
    """
Function to get the activations states from the reservoir
    :param data: train data
    :param resSize: reservoir size
    :param trainLen: length of train data
    :param initLen: inital data to be ignored
    :param Win: W in Matrix
    :param W: W matrix
    :param a: leaking rate
    :param X: matrix of Activations per input
    :return: last activation vector, matrix of activations X
    """
    x = zeros((resSize,1))
    for t in range(trainLen):
        u = data[t]
        x_u = tanh( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        x = (1-a)*x + a*x_u 
        if t >= initLen:
            X[:,t-initLen] = vstack((1,u,x))[:,0]
    return x,X

In [396]:
def train_output_l2(inSize,resSize,X,Yt,reg = 1e-8):
    """
Function to train readout with ridge regression to address overfitting
    :param inSize: size of input
    :param resSize: reservoir size
    :param reg: penalization parameter
    :param X: Matrix of activations
    :param Yt: target values
    :return Wout: weights from reservoir and input to output layer
    """
    X_T = X.T 
    Wout = dot( dot(Yt,X_T), linalg.inv( dot(X,X_T) + reg*eye(1+inSize+resSize) ) )
    return Wout

In [397]:
def get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x):
    """
Function to get the predictions of the normalized data
    :param data: train data
    :param outSize: output size
    :param trainLen: length of train data
    :param testLen: length of test data
    :param a: leaking rate
    :param Win: W in Matrix
    :param W: W matrix
    :param Wout: W out Matrix
    :param x: last activations states
    :return: vector of predictions
    """
    Y = zeros((outSize,testLen))
    u = data[trainLen]
    for t in range(testLen):
        x = (1-a)*x + a*tanh( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        y = dot( Wout, vstack((1,u,x)) )
        Y[:,t] = y
        # generative mode, this take the prediction and use it to make other prediction:
        u = y
        ## this would be a predictive mode, this made a prediccion and save it, then use the true value to make a new prediction:
        #u = data[trainLen+t+1]
    return Y

In [13]:
def denormalize_predictions(d_mean,d_std,Y):
    """
Function to denormalize predictions
    :param d_mean: data mean
    :param d_std: data standard deviation
    :param Y: predictions
    :return: denormalized predictions
    """
    Y1 = (Y * d_std) + d_mean
    return Y1

In [14]:
def get_MSE(predictions, target):
    """
    Function to get MSE from diference between predictions and real value (ouptuts)
    :param predictions: predictions made by ffnn
    :param target: true values from output values
    :return: MSE
    """
    return  (np.sum((target - predictions)**2))/len(predictions)

In [15]:
def SolutionValidation(Predictions,MSE,bestESN,bestMSE,bestPredictions,currentSetup):
    """
Function to validate ESN model
    :param Predictions: Prediction of current ESN
    :param MSE: MSE of current ESN
    :param bestESN: best setup of ESN so far
    :param bestMSE: MSE of best ESN so far
    :param bestPredictions: predictions of best ESN so far
    :param currentSetup: current ESN setup
    :return: bestESN, bestPredictions, bestMSE
    """
    if((float)(MSE) < (bestMSE)):
        bestESN = currentSetup
        bestMSE = MSE
        bestPredictions = Predictions
    return bestESN,bestPredictions,bestMSE

In [16]:
def LocalSolutionValidation(Predictions,MSE,listl2_pen,bestESN,bestMSE,bestPredictions,best_listl2_pen,currentSetup):
    """
Function to validate Local ESN model
    :param Predictions: Prediction of current ESN
    :param MSE: MSE of current ESN
    :param bestESN: best setup of ESN so far
    :param bestMSE: MSE of best ESN so far
    :param bestPredictions: predictions of best ESN so far
    :param currentSetup: current ESN setup
    :return: bestESN, bestPredictions, bestMSE
    """
    if((float)(MSE) < (bestMSE)):
        bestESN = currentSetup
        bestMSE = MSE
        bestPredictions = Predictions
        best_listl2_pen = listl2_pen
    return bestESN,bestPredictions,bestMSE, best_listl2_pen

In [17]:
def init_best(numl2):
    best_setup = (0,0,0,0,0)
    best_mse = 100000
    best_pred = (0,0,0,0,0,0)
    list_l2pen = zeros((numl2,7))
    return best_setup,best_mse,best_pred,list_l2pen

In [18]:
def toCSVandPlot(folder,name,esn,mse,pred,TS):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with open(folder + name + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['ESN configuration: ',esn])
        spamwriter.writerow(['predictions'])
        spamwriter.writerow(pred)
        spamwriter.writerow(['MSE:' , mse])
        csvfile.close()        
    #plot_Solutions(folder,name,pred,TS)
    gc.collect()

In [19]:
def toCSV(folder,name,b_esn,mse,pred,c_esn,list,TS):
    if not os.path.isdir(folder):
        os.makedirs(folder) 
    with open(folder + name + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['ESN configuration: ',b_esn])
        spamwriter.writerow(['predictions'])
        spamwriter.writerow(pred)
        spamwriter.writerow(['MSE:' , mse])
        spamwriter.writerow(['Current Setup w_o l2 penalty',c_esn])
        spamwriter.writerow(['Predictions'])
        spamwriter.writerows(list)
        csvfile.close()        
    #plot_Solutions2(folder,name,pred,list,TS)
    gc.collect()

In [232]:
def writeIterations(listPredTuple):
    folder = "Results/iteraciones"  +"/" 
    with open(folder + 'Validation_predicions' + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['Param1','Param2','Param3','Param4','Param5','Pred1','Pred2','Pred3','Pred4','Pred5','Pred6','MSE','Time'])
        for x in listPredTuple:
                    y = np.append(np.append(x[0],x[2]),np.append(x[1],x[3]))
                    spamwriter.writerow(y)
    csvfile.close()

In [261]:
def writePredictions(listPredTuple):
    folder = "Results/iteraciones"  +"/" 
    with open(folder + 'Test_predicions' + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['Param1','Param2','Param3','Param4','Param5','Pred1','Pred2','Pred3','Pred4','Pred5','Pred6','MSE'])
        for x in listPredTuple:                
                    y = np.append(np.append(x[0],x[1]),x[2])
                    spamwriter.writerow(y)
    csvfile.close()

In [22]:
def plot_Solutions(folder,name,Pred,TS):
    """
    Function to Plot List of predictions, mean predictions and Original Time Serie
    :param folder: path of file
    :param name: name of file
    :param Pred: predictions
    :param TS: Original Time Serie
    :return: matplotlib fig
    """
    fig = figure()
    TSValOutputs = TS[len(TS)-7:len(TS)].copy()
    TSValOutputs.plot(style= 'ko-')
    
    TSPred = pd.Series(Pred.astype(float), index = TS.index[len(TS)-12:len(TS)-6])
    TSPrediction = TS[len(TS)-7:len(TS)].copy()
    TSPrediction[1:] = TSPred
    TSPrediction.plot(style= 'bo-')
    lineMean = mlines.Line2D([],[],color='blue', marker='o',label = 'Prediction')
    lineVal = mlines.Line2D([],[],color='black', marker='o',label = 'True Values')
    legend(handles = [lineMean,lineVal],loc= 1)
    fig.savefig(folder + name + ".png")
    fig.clf()
    del fig
    gc.collect()
    return None

In [23]:
def plot_Solutions2(folder,name,Pred,list,TS):
    """
    Function to Plot List of predictions, mean predictions and Original Time Serie
    :param folder: path of file
    :param name: name of file
    :param Pred: predictions
    :param TS: Original Time Serie
    :return: matplotlib fig
    """
    fig = figure()
    TSValOutputs = TS[len(TS)-7:len(TS)].copy()
    TSValOutputs.plot(style= 'ko-')
    
    TSPred = pd.Series(Pred.astype(float), index = TS.index[len(TS)-12:len(TS)-6])
    TSPrediction = TS[len(TS)-7:len(TS)].copy()
    TSPrediction[1:] = TSPred
    TSPrediction.plot(style= 'bo-')
    lineMean = mlines.Line2D([],[],color='blue', marker='o',label = 'Prediction')
    lineVal = mlines.Line2D([],[],color='black', marker='o',label = 'True Values')
    legend(handles = [lineMean,lineVal],loc= 1)
    fig.savefig(folder + name + ".png")
    fig.clf()
    del fig
    gc.collect()
    return None

In [24]:
def get_denormilze_prediction(data,d_mean,d_std,outSize,trainLen,testLen,a,Win,W,Wout,x):
    Y = get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x)
    Y = denormalize_predictions(d_mean,d_std,Y)
    return Y

In [25]:
def init_local_best(numl2):
    local_best_setup = (0,0,0,0,0)
    local_best_mse = 100000
    local_best_pred = (0,0,0,0,0,0)
    local_list_l2pen = zeros((numl2,7))
    return local_best_setup,local_best_mse,local_best_pred,local_list_l2pen

In [26]:
def get_best_setup(ParameterList):
    """
Function to test several ESN model for the best setup
    :param ParameterList: List of vectors for different values of the parameters in this order:
    resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
    """    
    start = time.time()
    random.seed()
    data,target,d_mean,d_std = load_data()
    trainLen =len(data) - 6
    testLen = 6
    initLen = 12
    inSize = outSize = 1
    TS = get_TS()
    time_stamp = time.strftime("%d-%m") + "_" + time.strftime("%H-%M")
    folder = "Results/iteraciones"  +"/" 
    best_setup,best_MSE,best_predictions,listl2_pen = init_best(len(ParameterList[4]))
    param1 = 0   
    for resSize in ParameterList[0]:
       # name1 = ((str)(param1) + '-')
        param1 += 1
        param2 = 0
        for WinScale in ParameterList[1]:
            #name2 = name1 + (str)(param2) + '-'
            param2 += 1
            param3 = 0
            Win = generate_win_matrix(inSize,resSize,WinScale)
            for SRS in ParameterList[2]:
                #name3 = name2 + (str)(param3) + '-'
                param3 += 1
                param4 = 0
                W = generate_w_matrix(resSize)            
                rhoW = get_spectral_radius(W)
                W = scale_w(W,rhoW,SRS)
                X,Yt = get_x_yt_matrix(data,inSize,resSize,trainLen,initLen)
                for a in ParameterList[3]:
                    #name4 = name3 + (str)(param4) + '-'
                    param4 += 1
                    param5 = 0
                    x,X = get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X)
                    for reg in ParameterList[4]:
                        #name5 = name4 + (str)(param5)
                        param5 += 1
                        Wout = train_output_l2(inSize,resSize,X,Yt,reg)
                        Y = get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x)
                        Y = denormalize_predictions(d_mean,d_std,Y)
                        MSE = get_MSE(Y[0],target)
                        listl2_pen[param5-1]= (np.append(Y,MSE))
                        #add list of results for l2 penalty
                        currentSetup = (resSize,WinScale,SRS,a,reg)
                        gc.collect()
                        best_setup,best_predictions,best_MSE = SolutionValidation(Y[0],MSE,best_setup,best_MSE,best_predictions,currentSetup)
                  # toCSV(folder,name5,best_setup,best_MSE,best_predictions,currentSetup[:-1],listl2_pen,TS)
    stop = time.time()
    duration = stop - start
    toCSVandPlot(folder,('best-' + time_stamp) ,best_setup,best_MSE,best_predictions,TS)
    windows_explorer_path =  'C:\Users\sa-al\PycharmProjects\AnnEchoState\Results' 
    #os.startfile(windows_explorer_path)    
    return best_setup,best_MSE,best_predictions,duration

In [398]:
def get_best_ESN(ParameterList):
    """
Function to test several ESN model for the best setup
    :param ParameterList: List of vectors for different values of the parameters in this order:
    resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
    """
    start = time.time()
    random.seed()
    data, target, d_mean, d_std = load_data()

    # for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
    # be use as a target in the training, and we will use a second trainLen to choose the model
    #FulltrainLen = len(data) - 6
    trainLen = len(data) - 13

    # for choose the best model we have to evaluate in a certain number of predictios with know real values
    testLen = 6

    initLen = 12
    inSize = outSize = 1
    TS = get_TS()
    time_stamp = time.strftime("%d-%m") + "_" + time.strftime("%H-%M")
    folder = "Results/iteraciones" + "/"
    best_setup, best_MSE, best_predictions, listl2_pen = init_best(len(ParameterList[4]))
    bWin = 0
    bW = 0
    bx =0
    bX = 0
    
    #param1 = 0
    for resSize in ParameterList[0]:
        # name1 = ((str)(param1) + '-')
        #param1 += 1
        #param2 = 0
        
        X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)
        
        for WinScale in ParameterList[1]:
            # name2 = name1 + (str)(param2) + '-'
            #param2 += 1
            #param3 = 0

            Win = generate_win_matrix(inSize, resSize, WinScale)
            for SRS in ParameterList[2]:
                # name3 = name2 + (str)(param3) + '-'
                #param3 += 1
                #param4 = 0
                W = generate_w_matrix(resSize)
                rhoW = get_spectral_radius(W)
                W = scale_w(W, rhoW, SRS)

                # this will create the state matrix of 167 rows and a Yt vector with the last value
                # X,Yt = get_x_yt_matrix(data,inSize,resSize,FulltrainLen,initLen)
                # we create a X state matrix and Yt for training data
                # X_train,Yt_train = get_x_yt_matrix(data,inSize,resSize,trainLen,initLen)

                for a in ParameterList[3]:
                    # name4 = name3 + (str)(param4) + '-'
                    #param4 += 1
                    #param5 = 0

                    # this will fill the state matrix with all the data   
                    x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)

                    # using train and validation data to select best model
                    for reg in ParameterList[4]:
                        # name5 = name4 + (str)(param5)
                        #param5 += 1

                        # Train the output with the data to -6 observation in order to select the best model to make predictions
                        Wout = train_output_l2(inSize, resSize, X, Yt, reg)
                        # we get the predictions with training data
                        Y = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)

                        Y = denormalize_predictions(d_mean, d_std, Y)
                        MSE = get_MSE(Y[0], target)
                        # ESN = [a,Win,W,Wout,x_train]
                        currentSetup = (resSize, WinScale, SRS, a, reg)
                        gc.collect()

                        #bbest_MSE = best_MSE

                        best_setup, best_predictions, best_MSE = SolutionValidation(Y[0], MSE, best_setup, best_MSE,
                                                                                    best_predictions, currentSetup)

                        # if best_MSE != bbest_MSE:
                        #     bWin = Win
                        #     bW = W
                        #     bx = x
                        #     bX = X

    # Once we have the best setup we use the test values
    # We retrain de model with all the data, but for the train the last row of the state matrix shouldn't be used, this due we don't have
    # the target value for this row (this is the last observation, it will be use for the prediction

    #Wout = train_output_l2(inSize, best_setup[0], bX, Yt, best_setup[4])
    # with the model re trained we made the predictions for test data
    #Ytest = get_predictions(data, outSize, trainLen, testLen, best_setup[3], bWin, bW, Wout, bx)
    #Ytest = denormalize_predictions(d_mean, d_std, Ytest)
    stop = time.time()
    duration = stop - start
    # toCSVandPlot(folder,('best-' + time_stamp) ,best_setup,best_MSE,best_predictions,TS)
    windows_explorer_path = 'C:\Users\sa-al\PycharmProjects\AnnEchoState\Results'
    # os.startfile(windows_explorer_path)    
    return best_setup, best_MSE, best_predictions, duration
    # return ESN,data

In [28]:
#ParameterList_0_3 = [[100,200,300,400,500],[1,1.2,1.5,1.8,2,2.5,3,3.5,4],[0.5,1,1.2,1.5,2,2.5,3,3.5],[0.1,0.2,0.3,0.4,0.5,0.6,0.7],[1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1]]

In [254]:
ParameterList = [[100,200,300,400],[0.5,1.2,1.5,1.8,2,2.5],[0.3,0.5,0.8,1,1.5],[0.1,0.2,0.3,0.4,0.5,0.6],[1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1e-0]]

In [86]:
listPredTuple = list()
#for x in range(5):
 #  bestPredtuple = get_best_setup(ParameterList)   
  # listPredTuple.append(bestPredtuple)
#writeIterations(listPredTuple)

Aprox 1.- probar datos en mejor red

In [424]:
ParameterList = [[100],[0.5,1.2,1.5,1.8,2,2.5],[0.3,0.5,0.8,1,1.5],[0.1,0.2,0.3,0.4,0.5,0.6],[1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1e-0,1e1]]

In [399]:
predVal = list()
for x in range(1):
   bestPredtuple = get_best_ESN(ParameterList)  
   predVal.append(bestPredtuple)
writeIterations(predVal)

In [423]:
predVal[0][1]

In [408]:
predTest = list()
data, target, d_mean, d_std = load_dataTest()

#for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
#be use as a target in the training, and we will use a second trainLen to choose the model
trainLen = len(data) - 7

#for choose the best model we have to evaluate in a certain number of predictios with know real values
for solution in predVal:
    setup = solution[0]    
    testLen = 6
    
    initLen = 12
    inSize = outSize = 1
    #for solution in predVal:
    #setup = solution[0]
    resSize = setup[0]
    WinScale = setup[1]
    
    Win = generate_win_matrix(inSize, resSize, WinScale)
    SRS = setup[2]
    
    W = generate_w_matrix(resSize)
    rhoW = get_spectral_radius(W)
    W = scale_w(W, rhoW, SRS)
    
    #this will create the state matrix of 167 rows and a Yt vector with the last value
    X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)
    
    a = setup[3]
    
    #this will fill the state matrix with all the data   
    x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)
    
    #using train and validation data to select best model
    reg = setup[4]  #Train the output with the data to -6 observation in order to select the best model to make predictions
    Wout = train_output_l2(inSize, resSize, X, Yt, reg)
    #we get the predictions with training data
    Y = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)
    Y = denormalize_predictions(d_mean, d_std, Y)
    MSE = get_MSE(Y[0], target)
    test = (solution[0], Y[0], MSE)
    predTest.append(test)
gc.collect()

In [282]:
writePredictions(predTest)

In [300]:
data, target, d_mean, d_std = load_dataTest()

#for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
#be use as a target in the training, and we will use a second trainLen to choose the model
trainLen = len(data) - 7

#for choose the best model we have to evaluate in a certain number of predictios with know real values
testLen = 6

initLen = 12
inSize = outSize = 1

resSize = setup[0]
WinScale = setup[1]

Win = generate_win_matrix(inSize, resSize, WinScale)
SRS = setup[2]

W = generate_w_matrix(resSize)
rhoW = get_spectral_radius(W)
W = scale_w(W, rhoW, SRS)

#this will create the state matrix of 167 rows and a Yt vector with the last value
X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)

a = setup[3]

#this will fill the state matrix with all the data   
x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)

#using train and validation data to select best model
reg = setup[4]   #Train the output with the data to -6 observation in order to select the best model to make predictions
Wout = train_output_l2(inSize, resSize, X, Yt, reg)
#we get the predictions with training data
Y = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)
Y = denormalize_predictions(d_mean, d_std, Y)
MSE = get_MSE(Y[0], target)
gc.collect()

7

In [319]:
X[1]

array([  1.44619471e+00,  -7.87161629e-01,   1.56261641e+00,
         6.65975260e-02,  -1.41001774e+00,  -1.82137442e+00,
        -1.56524668e+00,   3.80936124e-01,  -1.76898466e+00,
         5.65270487e-01,   3.69293954e-01,  -3.38938073e-01,
        -7.92982715e-01,  -3.35057349e-01,  -2.88488668e-01,
         1.71008390e+00,   3.18844549e-01,   7.84531361e-01,
         5.78853019e-01,   3.77055401e-01,   3.44069252e-01,
         3.55711422e-01,   9.31998852e-01,   2.99440932e-01,
         1.15125973e+00,   9.12595235e-01,   7.74829553e-01,
        -1.29553640e+00,  -7.31085176e-02,   2.29587910e-01,
        -5.58198947e-01,   3.74921003e-02,   1.06394345e+00,
         8.29159681e-01,   6.56467488e-01,   1.06200309e+00,
         2.39095683e-02,   7.80650638e-01,  -1.81167262e+00,
        -7.87161629e-01,   7.63187383e-01,   5.34224700e-01,
         1.08334707e+00,  -4.70882670e-01,   2.31159603e+00,
        -6.80441735e-01,   1.07364526e+00,   8.73788000e-01,
         9.57223554e-01,

0.78000000000000003

In [234]:
MSE

14.370320141241708

Verificar si se tiene la ultima actualizacion de estados para el ultimo valor, es decir, para diciembre, ya que el trainlen es len(data)-1 y son en total 167 datos de 180, por lo que se ignoran 13, 12 de inicio y muy posiblemente el ultimo 

In [166]:
"""
Function to test several ESN model for the best setup
:param ParameterList: List of vectors for different values of the parameters in this order:
resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
"""
data, target, d_mean, d_std = load_data()

#for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
#be use as a target in the training, and we will use a second trainLen to choose the model
trainLen = len(data) - 7

#for choose the best model we have to evaluate in a certain number of predictios with know real values
testLen = 6

initLen = 12
inSize = outSize = 1

resSize = setup[0]
WinScale = setup[1]

Win = generate_win_matrix(inSize, resSize, WinScale)
SRS = setup[2]

W = generate_w_matrix(resSize)
rhoW = get_spectral_radius(W)
W = scale_w(W, rhoW, SRS)

#this will create the state matrix of 167 rows and a Yt vector with the last value
X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)
#we create a X state matrix and Yt for training data
a = setup[3]

#this will fill the state matrix with all the data   
x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)

#using train and validation data to select best model
reg = setup[4]  #Train the output with the data to -6 observation in order to select the best model to make predictions
Wout = train_output_l2(inSize, resSize, X_train, Yt_train, reg)
#we get the predictions with training data
Y = get_predictions(data, outSize, trainLen, train_testLen, a, Win, W, Wout, x_train)
Y = denormalize_predictions(d_mean, d_std, Y)
MSE = get_MSE(Y[0], target)
gc.collect()

ValueError: negative dimensions are not allowed

In [126]:
Wout

array([[ 0.03732527, -0.00682248,  0.02258167, -0.01547646,  0.00178117,
        -0.01343747,  0.02259832,  0.01000192, -0.02660947,  0.01740042,
         0.02657468,  0.00632588,  0.01687636,  0.00630895, -0.01707165,
         0.03763292, -0.03332238, -0.02796827, -0.00497894,  0.01215788,
         0.0389342 ,  0.02073109,  0.01601968,  0.023575  , -0.00964526,
         0.01180235, -0.0113277 , -0.01029719,  0.0154769 , -0.01141954,
         0.00284216,  0.02220816,  0.03795495, -0.02445965, -0.0144615 ,
         0.0538265 , -0.0143274 , -0.00979557, -0.0148354 , -0.00943266,
         0.00101309,  0.01242093,  0.01338527,  0.00532372,  0.02635308,
         0.00346995,  0.01957565, -0.00654978,  0.04168019,  0.00169869,
         0.04616836,  0.00071029,  0.02435967, -0.00409308, -0.03962271,
        -0.03204535, -0.01056922, -0.03074471, -0.00040139,  0.01215734,
         0.02137143, -0.02331153,  0.01969314, -0.0141678 ,  0.02437525,
         0.00173053,  0.00814639,  0.01821481,  0.0

In [43]:
#Wout = train_output_l2(inSize,best_setup[0],X,Yt,best_setup[4]) 
#with the model re trained we made the predictions for test data
Ytest = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)
Ytest = denormalize_predictions(d_mean, d_std, Ytest)

0.78000000000000003

In [42]:
#Train the output with the data to -6 observation in order to select the best model to make predictions
#Wout = train_output_l2(inSize, resSize, X_train, Yt_train, reg)
#we get the predictions with training data
Y = get_predictions(data, outSize,trainLen, testLen, a, Win, W, Wout, x)

Y = denormalize_predictions(d_mean, d_std, Y)

In [38]:
Y[0]

array([  -5.52162462,   -4.07464335,    4.62274111,    6.73633344,
        -19.16466122,  254.55997074])

In [112]:
Y[0]

array([ -1.89018221e+00,  -2.46552070e+00,   2.34024111e-01,
        -7.14257141e+00,   2.74749987e+01,   3.26825322e+03])

In [121]:
X[:,:-1].shape

(102, 167)